In [1]:
import warnings
warnings.filterwarnings('ignore')
import torch
import os
from pprint import pprint
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI

from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent

import tool_pool as tp

dotenv_path = '/Users/minkyuramen/Desktop/project/env'
load_dotenv(dotenv_path)

api_key = os.getenv("SEMANTIC_SCHOLAR_API_KEY")
openai_key = os.getenv("OPENAI_API_KEY")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /c1/yumin/.cache/huggingface/token
Login successful
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /c1/yumin/.cache/huggingface/token
Login successful


In [2]:
# # load model
# model = ChatOpenAI(model="gpt-3.5-turbo")
# load tool
tools = [tp.loadpaper, tp.recommendpaper, tp.loadfigure]
# load Agent prompt
prompt = hub.pull("hwchase17/openai-tools-agent")

In [3]:
# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# loadpaper

In [ ]:
output = agent_executor.invoke({"input": "explain about the math expression about the Filtering api calls in the 3 page of the paper 'Language Models Can Teach Themselves to Use Tools' 한글로 말해줘"})

In [ ]:
pprint(output['output'])

In [ ]:
output = agent_executor.invoke({"input": "summary the method of the paper 'Large Language Model Connected with Massive APIs' 한글로 말해줘"})

In [ ]:
pprint(output['output'])

# recompendpaper

In [ ]:
output = agent_executor.invoke({"input": "recommend the future works of the paper 'Facilitating Large Language Models to Master 16000+ Real-world APIs' 한글로 말해줘"})

In [ ]:
pprint(output['output'])

In [ ]:
output = agent_executor.invoke({"input": "Could you recommend previous works to read before this papaer : 'ToolLLM: Facilitating Large Language Models to Master 16000+ Real-world APIs' 한글로 말해줘"})

In [ ]:
pprint(output['output'])

# loadfigure

In [ ]:
output = agent_executor.invoke({"input": "explain about the 'Figure 2: Pass Rate evaluation in ToolBench paper.' in the 3 page of the paper 'StableToolBench' 한글로 말해줘"})

In [ ]:
pprint(output['output'])

## Memory in Agent

In [5]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory
from langchain_community.chat_message_histories import RedisChatMessageHistory

# prompt = hub.pull("hwchase17/react")
prompt = hub.pull("hwchase17/openai-tools-agent")
memory = ChatMessageHistory(session_id="test-session")
# message_history = RedisChatMessageHistory(
#     url="redis://127.0.0.1:6379/0", ttl=600, session_id="my-session"
# )

In [6]:
memory

InMemoryChatMessageHistory(messages=[])

In [7]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import OpenAI

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [8]:
output = agent_with_chat_history.invoke(
    {"input": "Summary the abstract of the paper 'EFFICIENT STREAMING LANGUAGE MODELS WITH ATTENTION SINKS'"},
    config={"configurable": {"session_id": "<test_session_id>"}},
)



> Entering new AgentExecutor chain...

Invoking: `loadpaper` with `{'title': 'EFFICIENT STREAMING LANGUAGE MODELS WITH ATTENTION SINKS', 'sections': []}`


Section list was incorrect.
Here is the title and section of the paper
title
Efficient Streaming Language Models 
with Attention Sinks
sections
          Abstract
  1 Introduction
  2 Related Work
  3 StreamingLLM
    3.1 The Failure of Window Attention and Attention Sinks
      Identifying the Point of Perplexity Surge.
      Why do LLMs break when removing initial tokens’ KV?
      LLMs attend to Initial Tokens as Attention Sinks.
    3.2 Rolling KV Cache with Attention Sinks
    3.3 Pre-Training LLMs with Attention Sinks
  4 Experiments
    4.1 Language Modeling on Long Texts Across LLM Families and Scales
    4.2 Results of Pre-Training with a Sink Token
      Convergence and Normal Model Performance.
      Streaming Performance.
      Attention Visualization.
    4.3 Results on Streaming Question Answering with Instruction-tu

In [31]:
output = agent_with_chat_history.invoke(
    {"input": "저 논문에서 \n 토큰을 4번 넣으면서 확인하고 싶었던 게 뭐야?"},
    config={"configurable": {"session_id": "<test_session_id>"}},
)



> Entering new AgentExecutor chain...

Invoking: `loadpaper` with `{'title': 'EFFICIENT STREAMING LANGUAGE MODELS WITH ATTENTION SINKS'}`


Here is the title and section of the paper
title
Efficient Streaming Language Models 
with Attention Sinks
sections
          Abstract
  1 Introduction
  2 Related Work
  3 StreamingLLM
    3.1 The Failure of Window Attention and Attention Sinks
      Identifying the Point of Perplexity Surge.
      Why do LLMs break when removing initial tokens’ KV?
      LLMs attend to Initial Tokens as Attention Sinks.
    3.2 Rolling KV Cache with Attention Sinks
    3.3 Pre-Training LLMs with Attention Sinks
  4 Experiments
    4.1 Language Modeling on Long Texts Across LLM Families and Scales
    4.2 Results of Pre-Training with a Sink Token
      Convergence and Normal Model Performance.
      Streaming Performance.
      Attention Visualization.
    4.3 Results on Streaming Question Answering with Instruction-tuned Models
    4.4 Ablation Studies
      Nu

In [34]:
output = agent_with_chat_history.invoke(
    {"input": "이 논문의 후속 연구를 알려줘 최신순으로"},
    config={"configurable": {"session_id": "<test_session_id>"}},
)



> Entering new AgentExecutor chain...

Invoking: `recommendpaper` with `{'query': 'EFFICIENT STREAMING LANGUAGE MODELS WITH ATTENTION SINKS', 'type': 'citation'}`


[{'paperId': '4ea5ca620122e6a9a2b000444d36491cebf49c7c', 'title': 'Advancing Transformer Architecture in Long-Context Large Language Models: A Comprehensive Survey', 'abstract': 'Transformer-based Large Language Models (LLMs) have been applied in diverse areas such as knowledge bases, human interfaces, and dynamic agents, and marking a stride towards achieving Artificial General Intelligence (AGI). However, current LLMs are predominantly pretrained on short text snippets, which compromises their effectiveness in processing the long-context prompts that are frequently encountered in practical scenarios. This article offers a comprehensive survey of the recent advancement in Transformer-based LLM architectures aimed at enhancing the long-context capabilities of LLMs throughout the entire model lifecycle, from pre-training t

### Real Use

In [8]:
from IPython.display import Markdown, display

In [9]:
def Paper_assistant(message, session_id = 'temp'):
    output = agent_with_chat_history.invoke(
        {"input": message},
        config={"configurable": {"session_id": session_id}},
    )
    display(Markdown(output['output']))
    return output

In [10]:
Paper_assistant('PROMPTAGENT: STRATEGIC PLANNING WITH LANGUAGE MODELS ENABLES EXPERT-LEVEL PROMPT OPTIMIZATION 후속 논문 추천해줘')



> Entering new AgentExecutor chain...

Invoking: `recommendpaper` with `{'query': 'PROMPTAGENT: STRATEGIC PLANNING WITH LANGUAGE MODELS ENABLES EXPERT-LEVEL PROMPT OPTIMIZATION', 'type': 'citation'}`




KeyError: 'abstract'

In [12]:
Paper_assistant('"Learning to Evolve via Policy-Level Reflection and Optimization" 논문 초록 요약해')



> Entering new AgentExecutor chain...

Invoking: `loadpaper` with `{'title': 'Agent-Pro: Learning to Evolve via Policy-Level Reflection and Optimization', 'sections': []}`




IndexError: list index out of range

In [3]:
from getpaper import GetPaper
import os

ss_api_key = os.getenv("SEMANTIC_SCHOLAR_API_KEY")


# Updated loadpaer tool
# model can use this tool several times, to get the list of the section, and then see the detail content of the paper.
# if the ar5iv_mode is False, 
getpapermodule = GetPaper(ss_api_key, ar5iv_mode = True, path_db = './papers_db', page_limit = 5)

In [ ]:
getpapermodule.load_paper(title = "Agent-Pro: Learning to Evolve via Policy-Level Reflection and Optimization")

IndexError: list index out of range